In [102]:
import requests, xmltodict, pandas as pd
from pandas import json_normalize
from zeep import Client, helpers
from datetime import datetime
import ast
import os
from pathlib import Path
import re
from typing import Optional
import re
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

In [2]:
def safe_serialize(obj):
    # tipos base
    if obj is None or isinstance(obj, (str, int, float, bool)):
        return obj
    if isinstance(obj, list):
        return [safe_serialize(x) for x in obj]
    if isinstance(obj, dict):
        return {k: safe_serialize(v) for k, v in obj.items()}

    # intentar con zeep.helpers.serialize_object (si está disponible)
    try:
        from zeep.helpers import serialize_object
        ser = serialize_object(obj)
        if ser is not None:
            return safe_serialize(ser)  # recursivo por si trae más objetos
    except Exception:
        pass

    # objetos Zeep suelen tener __values__
    vals = getattr(obj, "__values__", None)
    if isinstance(vals, dict):
        return {k: safe_serialize(v) for k, v in vals.items()}

    # fallback genérico
    try:
        return {k: safe_serialize(v) for k, v in vars(obj).items()}
    except Exception:
        return obj

def listar_ops(wsdl_url):
    c = Client(wsdl=wsdl_url)
    for svc in c.wsdl.services.values():
        for port in svc.ports.values():
            binding = port.binding
            for name, op in binding._operations.items():
                # Nombre de la operación
                args = []
                if op.input and op.input.body and op.input.body.type:
                    args = [elt[0] for elt in op.input.body.type.elements]  # nombres de parámetros
                print(f"{name}({', '.join(args)})")

In [9]:
listar_ops("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")

retornarPeriodosLegislativos()
retornarPeriodoLegislativoActual()
retornarLegislaturas()
retornarLegislaturaActual()
retornarMensajesXAnno(prmAnno)
retornarMocionesXAnno(prmAnno)
retornarMaterias()
retornarTramitesConstitucionales()
retornarTramitesReglamentarios()
retornarVotacionesXProyectoLey(prmNumeroBoletin)
retornarVotacionesXAnno(prmAnno)
retornarVotacionDetalle(prmVotacionId)
retornarProyectoLey(prmNumeroBoletin)
retornarProyectosLeyxNumeroLey(prmNumeroLey)
retornarPeriodosLegislativos()
retornarPeriodoLegislativoActual()
retornarLegislaturas()
retornarLegislaturaActual()
retornarMensajesXAnno(prmAnno)
retornarMocionesXAnno(prmAnno)
retornarMaterias()
retornarTramitesConstitucionales()
retornarTramitesReglamentarios()
retornarVotacionesXProyectoLey(prmNumeroBoletin)
retornarVotacionesXAnno(prmAnno)
retornarVotacionDetalle(prmVotacionId)
retornarProyectoLey(prmNumeroBoletin)
retornarProyectosLeyxNumeroLey(prmNumeroLey)
retornarPeriodosLegislativos()
retornarPeriodoLegislativoAct

In [18]:
c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL")
res = c.service.retornarPeriodosLegislativos()
print(res)

[{
    'Id': 1,
    'Nombre': '1990-1994',
    'FechaInicio': datetime.datetime(1990, 3, 11, 0, 0),
    'FechaTermino': datetime.datetime(1994, 3, 10, 0, 0),
    'Legislaturas': {
        'Legislatura': [
            {
                'Id': 3,
                'Numero': 319,
                'FechaInicio': datetime.datetime(1990, 3, 11, 0, 0),
                'FechaTermino': datetime.datetime(1990, 5, 20, 23, 59, 59),
                'Tipo': {
                    '_value_1': 'Extraordinaria',
                    'Valor': 2
                }
            },
            {
                'Id': 4,
                'Numero': 320,
                'FechaInicio': datetime.datetime(1990, 5, 21, 0, 0),
                'FechaTermino': datetime.datetime(1990, 9, 17, 23, 59, 59),
                'Tipo': {
                    '_value_1': 'Ordinaria',
                    'Valor': 1
                }
            },
            {
                'Id': 5,
                'Numero': 321,
                'Fec

In [19]:
d = safe_serialize(res) or {}
df = pd.json_normalize(d, sep=".")
df

,Id,Nombre,FechaInicio,FechaTermino,Diputados,Legislaturas.Legislatura,Legislaturas
0,1,1990-1994,1990-03-11,1994-03-10 00:00:00,None,"[{'Id': 3, 'Numero': 319, 'FechaInicio': 1990-...",NaN
1,2,1994-1998,1994-03-11,1998-03-10 00:00:00,None,"[{'Id': 12, 'Numero': 328, 'FechaInicio': 1994...",NaN
2,3,1998-2002,1998-03-11,2002-03-10 00:00:00,None,"[{'Id': 21, 'Numero': 337, 'FechaInicio': 1998...",NaN
3,4,2002-2006,2002-03-11,2006-03-10 00:00:00,None,"[{'Id': 30, 'Numero': 346, 'FechaInicio': 2002...",NaN
4,5,2006-2010,2006-03-11,2010-03-10 00:00:00,None,"[{'Id': 40, 'Numero': 356, 'FechaInicio': 2008...",NaN
5,6,2010-2014,2010-03-11,2014-03-10 00:00:00,None,"[{'Id': 42, 'Numero': 358, 'FechaInicio': 2010...",NaN
6,7,1965-1969,1965-03-11,1969-03-11 00:00:00,None,NaN,NaN
7,8,2014-2018,2014-03-11,2018-03-10 23:59:59,None,"[{'Id': 48, 'Numero': 364, 'FechaInicio': 2016...",NaN
8,9,2018-2022,2018-03-11,2022-03-10 23:59:59,None,"[{'Id': 50, 'Numero': 366, 'FechaInicio': 2018...",NaN
9,10,2022-2026,2022-03-11,2026-03-10 23:59:59,None,"[{'Id': 54, 'Numero': 370, 'FechaInicio': 2022...",NaN


In [13]:
# Carpeta raíz donde se crearán las subcarpetas
root = "data"

# Aseguramos que exista la carpeta principal
os.makedirs(root, exist_ok=True)

# Iteramos sobre cada fila del DataFrame
for _, row in df.iterrows():
    nombre_periodo = str(row["Nombre"])
    carpeta = os.path.join(root, nombre_periodo)
    os.makedirs(carpeta, exist_ok=True)

print("Carpetas creadas con éxito.")

Carpetas creadas con éxito.


### Save legislatures

In [20]:
df_explode = df.explode("Legislaturas.Legislatura").reset_index(drop=True)
df_legislaturas = json_normalize(df_explode["Legislaturas.Legislatura"])
df_legislaturas.head(5)

,Id,Numero,FechaInicio,FechaTermino,Tipo._value_1,Tipo.Valor
0,3.0,319.0,1990-03-11,1990-05-20 23:59:59,Extraordinaria,2.0
1,4.0,320.0,1990-05-21,1990-09-17 23:59:59,Ordinaria,1.0
2,5.0,321.0,1990-09-18,1991-05-20 23:59:59,Extraordinaria,2.0
3,6.0,322.0,1991-05-21,1991-09-17 23:59:59,Ordinaria,1.0
4,7.0,323.0,1991-09-18,1992-05-20 23:59:59,Extraordinaria,2.0


In [21]:
df_legislaturas.rename(columns={
    "Id" : "Legislatura.Id",
    "FechaInicio" : "Legislatura.FechaInicio",
    "FechaTermino" : "Legislatura.FechaTermino"
}, inplace=True)

df_legislaturas.head(5)

,Legislatura.Id,Numero,Legislatura.FechaInicio,Legislatura.FechaTermino,Tipo._value_1,Tipo.Valor
0,3.0,319.0,1990-03-11,1990-05-20 23:59:59,Extraordinaria,2.0
1,4.0,320.0,1990-05-21,1990-09-17 23:59:59,Ordinaria,1.0
2,5.0,321.0,1990-09-18,1991-05-20 23:59:59,Extraordinaria,2.0
3,6.0,322.0,1991-05-21,1991-09-17 23:59:59,Ordinaria,1.0
4,7.0,323.0,1991-09-18,1992-05-20 23:59:59,Extraordinaria,2.0


In [22]:
df_legislaturas = pd.concat([df_explode, df_legislaturas], axis=1)
df_legislaturas.drop(["Diputados", "Legislaturas.Legislatura", "Legislaturas", 
                      "FechaInicio", "FechaTermino", "Tipo.Valor"], axis=1, inplace=True)
df_legislaturas.rename(columns={"Nombre" : "Periodo"}, inplace=True)
df_legislaturas.head()

,Id,Periodo,Legislatura.Id,Numero,Legislatura.FechaInicio,Legislatura.FechaTermino,Tipo._value_1
0,1,1990-1994,3.0,319.0,1990-03-11,1990-05-20 23:59:59,Extraordinaria
1,1,1990-1994,4.0,320.0,1990-05-21,1990-09-17 23:59:59,Ordinaria
2,1,1990-1994,5.0,321.0,1990-09-18,1991-05-20 23:59:59,Extraordinaria
3,1,1990-1994,6.0,322.0,1991-05-21,1991-09-17 23:59:59,Ordinaria
4,1,1990-1994,7.0,323.0,1991-09-18,1992-05-20 23:59:59,Extraordinaria


In [54]:
BASE = Path("data")
BASE.mkdir(exist_ok=True)

def sanitize(name):
    """Limpia nombres de carpeta/archivo."""
    if pd.isna(name) or str(name).strip() == "":
        return "sin_periodo"
    return re.sub(r"[^\w\-]+", "_", str(name)).strip("_")

for col in ["Legislatura.FechaInicio", "Legislatura.FechaTermino"]:
    if col in df_legislaturas.columns:
        df_legislaturas[col] = pd.to_datetime(df_legislaturas[col], errors="coerce")

cols = [c for c in [
    "Legislatura.Id", "Numero",
    "Legislatura.FechaInicio", "Legislatura.FechaTermino",
    "Tipo._value_1", "Tipo.Valor"
] if c in df_legislaturas.columns]

# === 1) Split por periodo y guardar ===
for periodo, sub in df_legislaturas.groupby("Periodo", dropna=False):
    carpeta = BASE / sanitize(periodo)      # p.ej. data/1990-1994
    carpeta.mkdir(parents=True, exist_ok=True)

    # ordenar por fecha y número
    order_cols = [c for c in ["Legislatura.FechaInicio", "Numero"] if c in sub.columns]
    sub_sorted = sub.sort_values(order_cols) if order_cols else sub

    # seleccionar columnas, si no, guarda todo
    to_save = sub_sorted[cols] if cols else sub_sorted

    # guardar
    out_path = carpeta / "legislaturas.csv"
    to_save.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✓ Guardado {out_path}")

✓ Guardado data\1965-1969\legislaturas.csv
✓ Guardado data\1990-1994\legislaturas.csv
✓ Guardado data\1994-1998\legislaturas.csv
✓ Guardado data\1998-2002\legislaturas.csv
✓ Guardado data\2002-2006\legislaturas.csv
✓ Guardado data\2006-2010\legislaturas.csv
✓ Guardado data\2010-2014\legislaturas.csv
✓ Guardado data\2014-2018\legislaturas.csv
✓ Guardado data\2018-2022\legislaturas.csv
✓ Guardado data\2022-2026\legislaturas.csv


In [23]:
df_periodos = (
    df_legislaturas[["Id", "Periodo"]]
    .drop_duplicates()
    .set_index("Id")
)
df_periodos

,Periodo
Id,
1,1990-1994
2,1994-1998
3,1998-2002
4,2002-2006
5,2006-2010
6,2010-2014
7,1965-1969
8,2014-2018
9,2018-2022


### Save parliamentarians

In [64]:
import pandas as pd
from pandas import json_normalize
from zeep import Client

def estandarizar_df_diputados_x_periodo(client: Client, period_id: int, df_periodos: pd.DataFrame) -> pd.DataFrame:
    """
      - Mantiene sub-tramos de militancia dentro del período legislativo (cambios de partido).
      - Asigna periodo_id y Periodo por TRASLAPE con el período oficial (df_periodos).
      - Calcula militancias_anteriores (inicio < inicio_oficial_período).
      - Calcula edad al inicio oficial del período (o fallback 11-03).
      - Drop/rename y orden final de columnas para CSV.
    """

    # ===== 0) Normaliza df_periodos y saca la fila del período solicitado =====
    dfp = df_periodos.reset_index().copy()
    # Detecta columnas
    # ID
    cand_id = [c for c in dfp.columns if c in ["Id", "ID", "id", "periodo_id", "Id_periodo"]]
    if not cand_id:
        raise ValueError(f"df_periodos no tiene columna ID reconocible. Columnas: {list(dfp.columns)}")
    id_col = cand_id[0]
    # Label
    cand_label = [c for c in dfp.columns if c in ["Periodo", "Nombre", "NombrePeriodo", "periodo", "nombre"]]
    if not cand_label:
        raise ValueError(f"df_periodos no tiene columna de nombre/etiqueta de período. Columnas: {list(dfp.columns)}")
    label_col = cand_label[0]
    # Fechas (pueden o no existir)
    # Si faltan, derivamos (YYYY-YYYY) -> [YYYY-03-11, YYYY2-03-10]
    if "FechaInicio" in dfp.columns:
        dfp["FechaInicio"] = pd.to_datetime(dfp["FechaInicio"], errors="coerce")
    else:
        dfp["FechaInicio"] = pd.NaT
    if "FechaTermino" in dfp.columns:
        dfp["FechaTermino"] = pd.to_datetime(dfp["FechaTermino"], errors="coerce")
    else:
        dfp["FechaTermino"] = pd.NaT

    def _derive_dates_from_label(lbl):
        try:
            a, b = str(lbl).split("-")
            y1, y2 = int(a), int(b)
            ini = pd.Timestamp(f"{y1}-03-11")
            fin = pd.Timestamp(f"{y2}-03-10")
            return ini, fin
        except Exception:
            return pd.NaT, pd.NaT

    # Completa fechas faltantes desde el label
    mask_need_ini = dfp["FechaInicio"].isna()
    mask_need_fin = dfp["FechaTermino"].isna()
    if mask_need_ini.any() or mask_need_fin.any():
        der = dfp[label_col].apply(_derive_dates_from_label)
        dfp.loc[mask_need_ini, "FechaInicio"] = [t[0] for t in der[mask_need_ini]]
        dfp.loc[mask_need_fin, "FechaTermino"] = [t[1] for t in der[mask_need_fin]]

    # Saca período oficial solicitado
    rowp = dfp.loc[dfp[id_col] == period_id]
    if rowp.empty:
        raise ValueError(f"No encontré el período con Id={period_id} en df_periodos.")
    periodo_label_oficial = str(rowp.iloc[0][label_col])
    inicio_oficial = rowp.iloc[0]["FechaInicio"]
    fin_oficial = rowp.iloc[0]["FechaTermino"]

    # ===== 1) fetch + normalize del SOAP =====
    res = client.service.retornarDiputadosXPeriodo(period_id)
    d = safe_serialize(res) or {}
    df = pd.json_normalize(d, sep=".")

    # ===== 2) Explota militancias =====
    df_explode = df.explode("Diputado.Militancias.Militancia").reset_index(drop=True)

    # ===== 3) Normaliza objeto militancia =====
    df_militancias = (
        json_normalize(df_explode["Diputado.Militancias.Militancia"])
        .rename(columns={
            "FechaInicio": "Militancia.FechaInicio",
            "FechaTermino": "Militancia.FechaTermino",
        })
    )
    
    # ===== 4) Concat + fechas a datetime =====
    df_dip = pd.concat([df_explode, df_militancias], axis=1)
    for col in ["Militancia.FechaInicio", "Militancia.FechaTermino"]:
        if col in df_dip.columns:
            df_dip[col] = pd.to_datetime(df_dip[col], errors="coerce")

    # ===== 5) Mantener SOLO militancias que SOLAPAN con el período oficial =====
    SENTINEL_FUTURO = pd.Timestamp("2099-12-31")
    ini_mil = df_dip["Militancia.FechaInicio"]
    fin_mil = df_dip["Militancia.FechaTermino"].fillna(SENTINEL_FUTURO)
    solapa = (ini_mil <= fin_oficial) & (fin_mil >= inicio_oficial)
    df_dip = df_dip.loc[solapa].copy().set_index("Diputado.Id")

    # --- helper para nombre completo ---
    def _nombre_completo(row):
        partes = [
            row.get("Diputado.Nombre"),
            row.get("Diputado.Nombre2"),
            row.get("Diputado.ApellidoPaterno"),
            row.get("Diputado.ApellidoMaterno"),
        ]
        return " ".join([p for p in partes if p and str(p).strip() != ""]).strip()
    
    # ===== 6) Calcula nombre completo y asigna periodo oficial =====    
    df_dip["nombre_completo"] = df_dip.apply(_nombre_completo, axis=1)
    df_dip["Id_periodo"] = period_id
    df_dip["Periodo"] = periodo_label_oficial

    # ===== 7) militancias_anteriores: inicio < inicio_oficial (cuenta por diputado) =====
    # Ojo: contamos sobre TODO el set (no sólo los que solapan), por eso usamos el df original exploteado
    # pero en caso de querer sólo este subset, podrías usar df_dip en lugar de df_explode+militancias.
    # Aquí lo haremos con todas las militancias disponibles en la respuesta:
    df_all = pd.concat([df_explode, df_militancias], axis=1).set_index("Diputado.Id")
    if "Militancia.FechaInicio" in df_all.columns:
        df_all["Militancia.FechaInicio"] = pd.to_datetime(df_all["Militancia.FechaInicio"], errors="coerce")
    prev_counts = (
        df_all.loc[df_all["Militancia.FechaInicio"] < inicio_oficial]
              .groupby(level=0)
              .size()
              .rename("total_militancias_anteriores")
              .astype(int)
    )
    df_dip["total_militancias_anteriores"] = df_dip.index.map(prev_counts).fillna(0).astype(int)

    # ===== 8) EDAD al inicio oficial del período =====
    # Si no hay fecha oficial (NaT), fallback 11-03 del año inicial del label
    def _ref_date(label, fallback_ini):
        if pd.notna(inicio_oficial):
            return inicio_oficial
        try:
            y_ini = int(str(label).split("-")[0])
            return pd.Timestamp(f"{y_ini}-03-11")
        except Exception:
            return pd.NaT

    df_dip["Diputado.FechaNacimiento"] = pd.to_datetime(df_dip.get("Diputado.FechaNacimiento"), errors="coerce")
    ref_date = _ref_date(periodo_label_oficial, inicio_oficial)
    def _edad_en(dob, ref):
        if pd.isna(dob) or pd.isna(ref):
            return pd.NA
        return int(ref.year - dob.year - ((ref.month, ref.day) < (dob.month, dob.day)))
    df_dip["edad"] = df_dip["Diputado.FechaNacimiento"].apply(lambda dob: _edad_en(dob, ref_date))

    # ===== 9) DROP columnas que ya no interesan =====
    cols_drop = [
        "Distrito.Comunas.Comuna",
        "Diputado.Militancias.Militancia",
        "Diputado.FechaDefucion",
        "Diputado.RUT",
        "Diputado.RUTDV",
        "Partido.Alias",
        "FechaInicio",                 # del tramo 'res' si existiera
        "FechaTermino",                # del tramo 'res' si existiera
        "Diputado.Nombre",
        "Diputado.Nombre2",
        "Diputado.ApellidoPaterno",
        "Diputado.ApellidoMaterno",
        "Diputado.Sexo.Valor"
    ]
    df_dip.drop(columns=[c for c in cols_drop if c in df_dip.columns], errors="ignore", inplace=True)

    # ===== 10) RENOMBRES =====
    rename_map = {
        "Id_periodo": "periodo_id",
        "Diputado.FechaNacimiento": "fecha_nacimiento",
        "Diputado.Sexo._value_1": "sexo",
        "Distrito.Numero": "distrito_numero",
        "Partido.Id": "partido_id",
        "Partido.Nombre": "partido_nombre",
        "total_militancias_anteriores": "militancias_anteriores",
    }
    df_dip = df_dip.rename(columns=rename_map)

    # Asegura 'diputado_id' como columna
    df_dip["diputado_id"] = df_dip.index

    # ===== 11) Tipos útiles =====
    if "fecha_nacimiento" in df_dip.columns:
        df_dip["fecha_nacimiento"] = pd.to_datetime(df_dip["fecha_nacimiento"], errors="coerce")
    for int_col in ["periodo_id", "diputado_id", "distrito_numero", "militancias_anteriores"]:
        if int_col in df_dip.columns:
            if df_dip[int_col].isna().any():
                df_dip[int_col] = pd.to_numeric(df_dip[int_col], errors="coerce").astype("Int64")
            else:
                df_dip[int_col] = pd.to_numeric(df_dip[int_col], errors="coerce").astype(int)

    # ===== 12) Orden final recomendado =====
    ordered_cols = [
        "periodo_id",
        "Periodo",            # label oficial (para carpeta)
        "diputado_id",
        "nombre_completo",
        "fecha_nacimiento",
        "sexo",
        "edad",
        "distrito_numero",
        "partido_id",
        "partido_nombre",
        "militancias_anteriores",
        "Militancia.FechaInicio",
        "Militancia.FechaTermino"
    ]
    for c in ordered_cols:
        if c not in df_dip.columns:
            df_dip[c] = pd.NA

    df_final = df_dip[ordered_cols].reset_index(drop=True)
    return df_final


In [75]:
BASE = Path("data")
BASE.mkdir(exist_ok=True)

def sanitize(name):
    """Limpia nombres de carpeta/archivo."""
    if pd.isna(name) or str(name).strip() == "":
        return "sin_periodo"
    return re.sub(r"[^\w\-]+", "_", str(name)).strip("_")

# --- Aseguramos que df_periodos tenga columnas 'Id' y 'Periodo' ---
df_periodos_sorted = df_periodos.reset_index().sort_values("Id")

c = Client("https://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx?WSDL")

for _, row in df_periodos_sorted.iterrows():
    pid = int(row["Id"])                # ID del período legislativo
    nombre_periodo = str(row["Periodo"])  # Nombre o etiqueta del período, usado para carpeta

    try:
        df_pid = estandarizar_df_diputados_x_periodo(c, pid, df_periodos)

        carpeta = BASE / sanitize(nombre_periodo)   # p.ej., data/1990-1994
        carpeta.mkdir(parents=True, exist_ok=True)

        # (opcional) ordenar antes de guardar
        order_cols = [c for c in ["nombre_completo", "partido_nombre", "diputado_id"] if c in df_pid.columns]
        df_pid_sorted = df_pid.sort_values(order_cols) if order_cols else df_pid

        out_path = carpeta / "diputados.csv"
        df_pid_sorted.to_csv(out_path, index=False, encoding="utf-8")
        print(f"✓ Guardado {out_path}")

    except Exception as e:
        print(f"✗ Período {pid} ({nombre_periodo}): {e}")

✗ Período 1 (1990-1994): El servidor no puede procesar la solicitud. ---> La conversión del tipo 'DBNull' en el tipo 'Integer' no es válida.
✗ Período 2 (1994-1998): El servidor no puede procesar la solicitud. ---> La conversión del tipo 'DBNull' en el tipo 'Integer' no es válida.
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\1998-2002\diputados.csv
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\2002-2006\diputados.csv
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\2006-2010\diputados.csv
Index(['Militancia.FechaInicio', 'Militancia.FechaTermino', 'Partido.Id',
       'Partido.Nombre', 'Partido.Alias'],
      dtype='object')
✓ Guardado data\2010-2014\diputados.csv
✗ 

In [76]:
df = pd.read_csv("data\\2018-2022\\diputados.csv")

,periodo_id,Periodo,diputado_id,nombre_completo,fecha_nacimiento,sexo,edad,distrito_numero,partido_id,partido_nombre,militancias_anteriores,Militancia.FechaInicio,Militancia.FechaTermino
0,9,2018-2022,897,Alejandra Sepúlveda Orbenes,1965-11-13,Femenino,52,NaN,FRVS,Federación Regionalista Verde Social,4,2018-03-11 00:00:00,2022-03-10 23:59:59
1,9,2018-2022,1013,Alejandro Bernales Maldonado,1979-02-02,Masculino,39,NaN,LIBERAL,Partido Liberal de Chile,0,2018-03-11 00:00:00,2022-03-10 23:59:59
2,9,2018-2022,952,Alejandro Santana Tirachini,1965-07-18,Masculino,52,NaN,RN,Renovación Nacional,2,2018-03-11 00:00:00,2022-03-10 23:59:59
3,9,2018-2022,1076,Alexis Sepúlveda Soto,1969-12-29,Masculino,48,NaN,PR,Partido Radical de Chile,0,2018-03-11 00:00:00,2022-03-10 23:59:59
4,9,2018-2022,1043,Amaro Labra Sepúlveda,1949-10-25,Masculino,68,NaN,PC,Partido Comunista,0,2018-03-11 00:00:00,2022-03-10 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,9,2018-2022,991,Vlado Mirosevic Verdugo,1987-05-23,Masculino,30,NaN,LIBERAL,Partido Liberal de Chile,1,2018-03-11 00:00:00,2022-03-10 23:59:59
186,9,2018-2022,957,Víctor Torres Jeldes,1975-10-11,Masculino,42,NaN,DC,Partido Demócrata Cristiano,2,2018-03-11 00:00:00,2022-03-10 23:59:59
187,9,2018-2022,1059,Ximena Ossandón Irarrázabal,1963-12-13,Femenino,54,NaN,RN,Renovación Nacional,0,2018-03-11 00:00:00,2022-03-10 23:59:59
188,9,2018-2022,1017,Álvaro Carter Fernández,1980-10-09,Masculino,37,NaN,IND,Independientes,0,2020-09-29 00:00:00,2022-03-10 23:59:59


In [108]:
# --- 1) Helpers de normalización ---
def norm_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def pick_field(txt: str, label_regex: str):
    """
    Devuelve el valor que sigue a una etiqueta (ej: 'Partido', 'Región', etc.),
    cortando antes de barras verticales, bullets, saltos, o el inicio de otro campo.
    """
    # Permite :, -, – como separadores tras la etiqueta
    m = re.search(label_regex + r"\s*[:\-–]\s*([^|•\n\r]+)", txt, flags=re.I)
    if not m:
        # fallback si no hay separador claro: tomar lo que sigue hasta el próximo campo conocido
        m = re.search(label_regex + r"\s*(.*?)(?:\s+(?:Comit[eé]|Partido|Regi[oó]n|Circunscripci[oó]n)\b|$)", txt, flags=re.I)
    return norm_spaces(m.group(1)) if m else None

def parse_card_text(txt: str) -> dict:
    """
    txt de ejemplo:
    'Carmen Gloria Aravena Acuña Circunscripción 11 Región de La Araucanía Partido Independiente Comité Partido Renovación Nacional e Independientes'
    """
    t = norm_spaces(txt)

    # Nombre completo: todo lo que va antes de la primera etiqueta conocida
    # (Circunscripción|Región|Partido|Comité)
    m = re.match(r"^(.*?)\s+(?=(?:Circunscripci[oó]n|Regi[oó]n|Partido|Comit[eé])\b)", t, flags=re.I)
    nombre = norm_spaces(m.group(1)) if m else None

    circ = pick_field(t, r"Circunscripci[oó]n")
    region = pick_field(t, r"Regi[oó]n")
    partido = pick_field(t, r"Partido")
    comite  = pick_field(t, r"Comit[eé]")

    # Si 'Partido' aparece dos veces (p.ej. 'Partido Independiente' y luego 'Comité Partido ...'),
    # nos quedamos con la primera ocurrencia como partido real.
    # (El pick_field ya corta en la siguiente etiqueta, así que normalmente queda bien.)

    # Circunscripción: intenta extraer número si te sirve (pero guardamos el texto completo también)
    circ_num = None
    if circ:
        mnum = re.search(r"(\d+)", circ)
        circ_num = int(mnum.group(1)) if mnum else None

    return {
        "nombre_completo": nombre,
        "circunscripcion": circ,
        "circunscripcion_numero": circ_num,
        "region": region,
        "partido": partido,
        "comite": comite,
    }

# --- 2) Uso sobre cada div tarjeta ---
# Supongamos que ya tienes 'cards' con: soup.select("div.col-md-6.col-lg-3.col-12.mb-4")
def parse_cards(cards) -> pd.DataFrame:
    filas = []
    for card in cards:
        # quita posible basura y extrae texto "compacto"
        for bad in card(["script", "style"]): bad.extract()
        texto = card.get_text(" ", strip=True)
        filas.append(parse_card_text(texto))
    return pd.DataFrame(filas)

In [109]:
html = requests.get(URL, headers={"User-Agent":"Mozilla/5.0"}).text
soup = BeautifulSoup(html, "html.parser")

cards = soup.select("div.col-md-6.col-lg-3.col-12.mb-4")


df = parse_cards(cards)
df.head()

,nombre_completo,circunscripcion,circunscripcion_numero,region,partido,comite
0,Carmen Gloria Aravena Acuña,11,11,de La Araucanía,Independiente,Partido Renovación Nacional e Independientes
1,Pedro Araya Guerrero,3,3,de Antofagasta,P.P.D.,Partido Por la Democracia e Independientes
2,Karim Bianchi Retamales,15,15,de Magallanes y la Antártica Chilena,Independiente,Independiente I
3,Fabiola Campillai Rojas,7,7,Metropolitana de Santiago,Independiente,Mixto
4,Loreto Carvajal Ambiado,16,16,de Ñuble,P.P.D.,Partido Por la Democracia e Independientes
